# Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet, LassoLars
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor, RegressorChain

from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error

In [3]:
from sktime.transformations.panel.rocket import Rocket

In [4]:
from sklearn.decomposition import PCA

In [5]:
from variable_assignation import *
from load_functions import *

# Load data

In [6]:
df_data_death, df_data_prop = {}, {}
for variant in ['alpha', 'gamma', 'kappa', 'delta']:
    input_files_folder = f'../input_files/{variant}'
    df_data_death[variant] = load_obj(f'{input_files_folder}/df_prop_deaths_{variant}')
    df_data_prop[variant] = load_obj(f'{input_files_folder}/df_prop_{variant}')

all_variables = []
for var_list in variables_vars.values():
    all_variables += var_list

# Variables

In [7]:
target_variant_labels = ['alpha', 'gamma']
add_variant_curve = ['alpha', 'gamma', 'kappa', 'delta']

In [8]:
## ~0.7 variantgamma_transmissibility + variantgamma_imports_factor + variantgamma_cross_protection_prob
## ~0.5 variable_alpha + variantgamma_transmissibility_factor - GradientBoostingRegressor
## ~0.6 variable_alpha + variantgamma_severity_factor - GradientBoostingRegressor

# Data process

In [25]:
tuple_lables = []
for variant in target_variant_labels:
    tuple_lables.append(df_data_death[variant]['y'].tolist())
tuple_lables = tuple(tuple_lables)

In [32]:
y_labels = []
for labels in zip(*tuple_lables):
    y_labels.append(np.concatenate(labels))
y_labels = np.array(y_labels)[:,:8]

In [33]:
df_data = pd.DataFrame({})
for n_var, variant in enumerate(add_variant_curve):
    df_data[f'dim_{n_var}'] = df_data_death[variant]['dim_0']
    df_data[f'dim_{n_var + len(add_variant_curve)}'] = df_data_prop[variant]['dim_0']

## PCA label transformation

In [34]:
# Initialize PCA - keep enough components to explain, for example, 95% of variance
# pca = PCA(n_components=0.99999) # alpha[:,:3] 0.99999 -> 0.9198530859936276
pca = PCA(n_components=0.9999) 

# Fit and transform data
y_labels_pca = pca.fit_transform(y_labels)

In [35]:
n_train = 1950
X_train = df_data[:n_train]
y_train = np.array(y_labels[:n_train])
y_train_pca = np.array(y_labels_pca[:n_train])

X_test = df_data[n_train:]
y_test = np.array(y_labels[n_train:])
y_test_pca = np.array(y_labels_pca[n_train:])

In [36]:
print(f"{'original shape y_train:': <30}", y_train.shape)
print(f"{'original shape y_test:': <30}", y_test.shape)

print(f"{'transformed shape y_train:': <30}", y_train_pca.shape)
print(f"{'transformed shape y_test:': <30}", y_test_pca.shape)

original shape y_train:        (1950, 8)
original shape y_test:         (29, 8)
transformed shape y_train:     (1950, 5)
transformed shape y_test:      (29, 5)


In [37]:
# Predict on the test set using the best model
y_test_post = pca.inverse_transform(y_test_pca.reshape(np.shape(y_test_pca)))

for n in range(5):
    print(f' Test point {n} '.center(50, "#"))
    print(f"{'Test_pre:': <10}", y_test[n])
    print(f"{'Test_post:': <10}", y_test_post[n])
    print('\n')

################## Test point 0 ##################
Test_pre:  [0.55074271 1.41025391 4.98791504 0.95       0.81840077 2.09562988
 6.53723145 0.61318115]
Test_post: [0.55058151 1.41031655 4.98791503 0.95       0.81817617 2.09571754
 6.53723145 0.61318126]


################## Test point 1 ##################
Test_pre:  [0.49216361 1.26025391 8.55041504 0.95       0.5010973  1.28312988
 2.97473145 0.64443115]
Test_post: [0.49214531 1.26026102 8.55041504 0.95       0.5010718  1.28313984
 2.97473145 0.64443116]


################## Test point 2 ##################
Test_pre:  [0.57026908 1.46025391 3.80041504 0.95       0.75494008 1.93312988
 7.72473145 0.76943115]
Test_post: [0.57013513 1.46030596 3.80041504 0.95       0.75475344 1.93320272
 7.72473145 0.76943124]


################## Test point 3 ##################
Test_pre:  [0.45311088 1.16025391 1.42541504 0.95       0.62801869 1.60812988
 5.34973145 0.58193115]
Test_post: [0.45316432 1.16023314 1.42541504 0.95       0.62809315 1.6081008

## ROCKET Feature generation

In [16]:
num_kernels = 20000
rocket = Rocket(num_kernels=num_kernels, n_jobs=15)
rocket.fit(X_train)

Rocket(n_jobs=15, num_kernels=20000)

In [17]:
%%time
X_train_transform = rocket.transform(X_train)
X_test_transform = rocket.transform(X_test)

CPU times: user 1h 1min 22s, sys: 1.2 s, total: 1h 1min 23s
Wall time: 4min 22s


# Train model

## Regressor chain

In [38]:
# Create the ElasticNet regressor
regressor = LassoLars(alpha=1e-7, normalize=True)

# Define regression chain
chain = RegressorChain(base_estimator=regressor)
chain.fit(X_train_transform, y_train_pca)

RegressorChain(base_estimator=LassoLars(alpha=1e-07, normalize=True))

In [ ]:
# Predict on the test set using the best model
pca_pred = chain.predict(X_test_transform)
y_pred = pca.inverse_transform(pca_pred.reshape(np.shape(y_test_pca)))

# Predict on the test set using the best model
print('Score: ', chain.score(X_test_transform, y_test_pca))

In [ ]:
for n in range(10):
    print(f' Test point {n} '.center(50, "#"))
    print(f"{'Pred:': <10}", y_pred[n])
    print(f"{'Target:': <10}", y_test[n])
    print('\n')

## Grid Search

In [25]:
%%time
# Standardize the transformed features
# scaler = StandardScaler()
# X_train_transform = scaler.fit_transform(X_train_transform)
# X_test_transform = scaler.transform(X_test_transform)

# Create the ElasticNet regressor
regressor = LassoLars(normalize=True)

# Define the parameter grid for grid search
param_grid = {
    'alpha': np.linspace(1e-7,1e-6,1),
}
#0.001
# Perform grid search
grid_search = GridSearchCV(regressor, param_grid)
grid_search.fit(X_train_transform, y_train_pca)

# Print the best hyperparameters and the corresponding score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Get the best model and its hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Predict on the test set using the best model
pca_pred = best_model.predict(X_test_transform)
y_pred = pca.inverse_transform(pca_pred.reshape(np.shape(y_test_pca)))

# Predict on the test set using the best model
print('Score: ', best_model.score(X_test_transform, y_test_pca))

Best Hyperparameters: {'alpha': 1e-07}
Best Score: 0.6851986684269541
CPU times: user 2h 35min 24s, sys: 2h 55min 7s, total: 5h 30min 32s
Wall time: 31min 39s


In [72]:
for n in range(5):
    print(f' Test point {n} '.center(50, "#"))
    print(f"{'Pred:': <10}", y_pred[n])
    print(f"{'Target:': <10}", y_test[n])
    print('\n')

################## Test point 0 ##################
Pred:      [0.81782004 2.09453465 6.35445222 0.64142241]
Target:    [0.81840077 2.09562988 6.53723145 0.61318115]


################## Test point 1 ##################
Pred:      [0.50141582 1.28396368 3.88178775 0.69323248]
Target:    [0.5010973  1.28312988 2.97473145 0.64443115]


################## Test point 2 ##################
Pred:      [0.71890668 1.84112862 8.31081901 0.66517127]
Target:    [0.75494008 1.93312988 7.72473145 0.76943115]


################## Test point 3 ##################
Pred:      [0.63899826 1.63643507 5.45257725 0.65767805]
Target:    [0.62801869 1.60812988 5.34973145 0.58193115]


################## Test point 4 ##################
Pred:      [0.88431229 2.26484523 0.00916891 0.6758042 ]
Target:    [0.88186146 2.25812988 0.59973145 0.70693115]




In [ ]:
# %%time
# # Define the ML model with MultiOutputRegressor and ExtraTreesRegressor
# model = MultiOutputRegressor(ExtraTreesRegressor(random_state=0), n_jobs=-1)

# model.fit(X_train_transform, y_train_pca)

# # Predict on the test set using the best model
# y_pred_pca = model.predict(X_test_transform)

# # Evaluate the model using mean squared error
# mse = mean_squared_error(y_test_pca, y_pred_pca)
# print("Mean Squared Error:", mse)

# print('Score: ',model.score(X_test_transform, y_test_pca))
# print('####################')
# y_pred_pca = model.predict(X_test_transform)
# for n in range(20):
#     print('Pred:  ',y_pred_pca[n])
#     print('Target:',y_test_pca[n])
#     print('####################')

# from sklearn import linear_model
# reg = linear_model.LassoLars(alpha=0.000001, normalize=True)

# classifier = RegressorChain(reg)
# classifier.fit(X_train_transform, y_train)

# # ~0.6 - 3 parameters

# # from sklearn import linear_model
# # reg = linear_model.LassoLarsIC(criterion='bic', normalize=True)

# from sklearn.linear_model import LassoLarsCV
# # # reg = LassoLarsCV(cv=3, normalize=True)

# # from sklearn.linear_model import ElasticNet
# # reg = ElasticNet(alpha=0.1, l1_ratio=0.1, normalize=False, random_state=0)

# # from sklearn import linear_model
# # reg = linear_model.TweedieRegressor(alpha=0.1, max_iter=1500, link='identity')
# # # 0.37 - 4 parameters alpha=0.1, max_iter=1500, link='identity'

# from sklearn.linear_model import RANSACRegressor
# reg = RANSACRegressor(random_state=0, min_samples=0.99)

# # scaler = preprocessing.StandardScaler().fit(X_train_twe)
# # X_train_scaled = scaler.transform(X_train_twe)